Notebook to do curved roadway generation

## Changelog
- Friday, Jan 31
    - Creation
    - Toy track generation using way if 10025 from `DR_CHN_Merging_ZS.osm`
- Monday, Feb 3
    - Begin trajdata creation to read in the vehicle tracks

In [56]:
# usings
using AutomotiveDrivingModels
using AutoViz

In [ ]:
# function: append to curve
function append_to_curve!(target::Curve, newstuff::Curve)
    s_end = target[end].s
    for c in newstuff
        push!(target, CurvePt(c.pos, c.s+s_end, c.k, c.kd))
    end
    return target
end

In [ ]:
# functions: angle calculation
"""
    function append_headings
- Used create the angles and append them into the coordinates

# Examples
```julia
x_coods = [1089.07510, 1093.82626, 1101.19325, 1112.59899, 1123.96733, 1133.24150, 1146.47964]
y_coods = [936.31213, 936.92692,938.52419, 940.93865, 943.27882, 945.21039, 947.88488]
coods = hcat(x_coods,y_coods)
append_headings(coods)
```
"""
function append_headings(coordinates::Matrix{Float64})
    headings = ones(size(coordinates)[1])
    for i = 1:size(coordinates)[1]-1
        # need to make sure that math is right, and that bounds are kept
        tangent_vector = [coordinates[i+1,1]-coordinates[i,1], coordinates[i+1,2]-coordinates[i,2]]
        # @show tangent_vector
        current_heading = get_new_angle(tangent_vector)
        # @show current_heading
        headings[i] = current_heading
    end
    headings[end] = headings[end-1] # assume this is fine
    coordinates = hcat(coordinates, headings)
    return coordinates
end

"""
    function get_new_angle
- Does the actual angle calculation based on the x y coordinates
"""
function get_new_angle(tangent_vector::Array{Float64})
    # it might be a problem when we switch quadrants
    # use signs of tangent vector to get the quadrant of the heading 
    x = tangent_vector[1]
    y = tangent_vector[2]
    if x == 0. && y == 0.
        heading = 0.0
    elseif x == 0.
        heading = π/2 * sign(y) 
    elseif y == 0.
        heading = convert(Float64, π) # this could be either pi or -pi, but just go with pi
    elseif sign(x) == 1 && sign(y) == 1 # first quadrant
        heading = atan(y, x)
    elseif sign(x) == -1 && sign(y) == 1 # second quadrant
        heading = atan(y, x)
    elseif sign(x) == 1 && sign(y) == -1 # fourth quadrant
        heading = atan(y, x)
    elseif sign(x) == -1 && sign(y) == -1 # third quadrant
        heading = atan(y, x)
    end
    # bound_heading doesn't end up getting called cause Julia takes care of it apparently
    bound_heading(heading)

    return heading
end

"""
    function bound_heading
- Make the angle range from 0 to pi instead of going beyond
"""
function bound_heading(heading::Float64)
    if heading > π # send this to be just near -pi
        heading = -π + (heading - π)    # if heading is 3.15, then the new angle will be (-pi + (3.15-pi)) = -3.13
    elseif heading < -π # send this to be just near pi 
        heading = π + (heading + π)     # if heading is -3.15, then the new angle will be (pi + (-3.15+pi)) = 3.13
    end
    return heading
end

In [ ]:
# WBs example code
"""
- WBs example code
"""
function get_track()

    x_coods = [0., 5.]
    y_coods = [3., 3.]
    theta = [0., 0.]
    
    turn_length = 10
    new_theta = 0.
    incremental_angle = (pi/4) / turn_length
    radius = 1
    for i = 1:turn_length
        push!(theta, new_theta)
        push!(x_coods, x_coods[end] + radius * cos(new_theta))
        push!(y_coods, y_coods[end] + radius * sin(new_theta))
        new_theta -= (incremental_angle)
    end
#     plot(x_coods, y_coods)
    mid_coods = vcat(x_coods', y_coods', theta') # x coods in first row, y coods in second row, theta in third row
    
    
    first_cood = VecSE2(mid_coods[1,1], mid_coods[2,1], mid_coods[3,1])
    second_cood = VecSE2(mid_coods[1,2], mid_coods[2,2], mid_coods[3,2])
    radius = 0.01
    nsamples = 20

    track = gen_bezier_curve(first_cood, second_cood, radius, radius, nsamples)
    
    curve_radius = incremental_angle
    nsamples = 1000
    for i = 3:turn_length+2
        turn1 = VecSE2(mid_coods[1, i-1], mid_coods[2, i-1], mid_coods[3, i-1])
        turn2 = VecSE2(mid_coods[1, i], mid_coods[2, i], mid_coods[3, i])
        curve = gen_bezier_curve(turn1, turn2, curve_radius, curve_radius, nsamples)
        append_to_curve!(track, curve)
    end

    return track
end

In [ ]:
# function: get_track
"""
    function get_track()

- Generate a track based on x and y coordinates of the centerline
- Extracts angles as an intermediate step
"""
function get_track()
    # way 10025 for illustrate purposes
    x_coods = [1089.07510, 1093.82626, 1101.19325, 1112.59899, 1123.96733, 1133.24150, 1146.47964]
    y_coods = [936.31213, 936.92692,938.52419, 940.93865, 943.27882, 945.21039, 947.88488]

    coods = hcat(x_coods,y_coods)
    coods_app = append_headings(coods) # Append with the angle
    
    mid_coods = coods_app'
    
    first_cood = VecSE2(mid_coods[1,1], mid_coods[2,1], mid_coods[3,1])
    second_cood = VecSE2(mid_coods[1,2], mid_coods[2,2], mid_coods[3,2])
    radius = 0.01
    nsamples = 20

    track = gen_bezier_curve(first_cood, second_cood, radius, radius, nsamples)
    
    nsamples = 20
    for i = 3:7
        turn1 = VecSE2(mid_coods[1, i-1], mid_coods[2, i-1], mid_coods[3, i-1])
        turn2 = VecSE2(mid_coods[1, i], mid_coods[2, i], mid_coods[3, i])
        curve = gen_bezier_curve(turn1, turn2, radius, radius, nsamples)
        append_to_curve!(track, curve)
    end

    return track
end

In [ ]:
# script: generate the roadway

#nlanes = 1
#length = 50.

#roadway = gen_straight_roadway(nlanes, length)

width = DEFAULT_LANE_WIDTH
roadway = Roadway()

track = get_track()

lane = Lane(LaneTag(1, 2), track, width=DEFAULT_LANE_WIDTH)
push!(roadway.segments, RoadSegment(lane.tag.segment, [lane]))
cam = FitToContentCamera(0.05)
render(roadway, cam=cam)

In [ ]:
@show roadway.segments[1].lanes

## Interaction track experimentation

In [ ]:
# usings: Specific to trajdata
using DataFrames
using Records

In [81]:
# constants: timestep
const INTERACTION_TIMESTEP = 0.1 # unit is second

0.1

In [37]:
# struct: the interaction trajdata
"""
INTERACTIONTrajdata
The trajectory data stored in the original INTERACTION dataset format.
The dataset is a csv file with columns:
    track_id      - Int64 - Representing the id of the agent
    frame_id      - Int64 - Represents the frames in which the agent appears in the video
    timestamp_ms  - Int64 - represents the time the agent appears in the video. The unit is millisecond
    agent_type    - String - Can be person, car, truck and so on
    x             - Float64 - x position, in meter
    y             - Float64 - y position in meter
    vx            - Float64 - x velocity in m/s
    vy            - Float64 - y velocity in m/s
    psi_rad       - Float64 - yaw angle in radian
    length        - Float64 - Length of the vehicle, in meter
    width         - Float64 - Width of the vehicle, in meter

# Example
```julia
tdraw = INTERACTIONTrajdata("vehicle_tracks_000.csv");
```
"""
mutable struct INTERACTIONTrajdata
    df         :: DataFrame
    car2start  :: Dict{Int, Int}         # maps carindex to starting index in the df
    frame2cars :: Dict{Int, Vector{Int}} # maps frame to list of carids in the scene

    function INTERACTIONTrajdata(input_path::String)

        @assert(isfile(input_path))

        df = readtable(input_path, separator=',', header = true)

        car2start = Dict{Int, Int}()
        frame2cars = Dict{Int, Vector{Int}}()

        for (dfind, carid) in enumerate(df[:track_id])
            if !haskey(car2start, carid)
                car2start[carid] = dfind
            end

            frame = convert(Int, df[dfind, :frame_id])
            if !haskey(frame2cars, frame)
                frame2cars[frame] = [carid]
            else
                frame2cars[frame] = push!(frame2cars[frame], carid)
            end
        end

        new(df, car2start, frame2cars)
    end
end

INTERACTIONTrajdata

In [47]:
# function: overload nframes from Records
"""
    Records.nframes

# Example
```julia
tdraw = INTERACTIONTrajdata("vehicle_tracks_000.csv")
nframes(tdraw)
```
"""
Records.nframes(trajdata::INTERACTIONTrajdata) = maximum(keys(trajdata.frame2cars))

In [39]:
tdraw = INTERACTIONTrajdata("vehicle_tracks_000.csv");

In [79]:
df = tdraw.df
vehdefs = Dict{Int, VehicleDef}()
states = Array{RecordState{VehicleState, Int}}(undef, nrow(df))
frames = Array{RecordFrame}(undef, nframes(tdraw));

In [80]:
# script: initialize the vehicle definition in terms of type, length and width
for (id, dfind) in tdraw.car2start
    # CAUTION: Hardcoding vehicle type to be just car shown by the first argument being 2
    vehdefs[id] = VehicleDef(2, df[dfind, :length], df[dfind, :width])
    #if df[dfind,:agent_type] == "car"
    #    print("dfind is $(dfind)")
    #    print(df[dfind,:agent_type])
    #end
end

In [ ]:
# script: loop over data and create the vehicle state sequence. NOTE: THIS NEEDS THE ROADWAY
state_ind = 0
for frame in 1 : nframes(tdraw)

    frame_lo = state_ind+1

    for id in carsinframe(tdraw, frame)
        dfind = car_df_index(tdraw, id, frame)

        posG = VecSE2(df[dfind, :x], df[dfind, :y], df[dfind, :psi_rad])
        speed = df[dfind, :speed]

        states[state_ind += 1] = RecordState(VehicleState(posG, roadway, speed), id)
    end

    frame_hi = state_ind
    frames[frame] = RecordFrame(frame_lo, frame_hi)
end
Trajdata(INTERACTION_TIMESTEP, frames, states, vehdefs)